In [ ]:
from mriqc_learn.datasets import load_dataset

(train_x, train_y), (_, _) = load_dataset(dataset="SHIP183", split_strategy="none")
train_x["site"] = train_y.site

train_y = train_y[["rating"]].values.squeeze().astype(int)
print(f'Excluded={100 * (train_y < 1).sum() / len(train_y)}')
print(f'Accept={100 * (train_y >= 1).sum() / len(train_y)}')
train_y[train_y >= 1] = 1

In [2]:
from pkg_resources import resource_filename as pkgrf
from joblib import load
from mriqc_learn.models import preprocess as pp
from sklearn.pipeline import Pipeline
import pandas as pd

# Load the trained model
model = load(pkgrf("mriqc_learn.data", "classifier.joblib"))


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:

# load the new dataset
X_new = pd.read_csv(
    pkgrf("mriqc_learn.data", "SHIP183.tsv"),
    sep="\t",
    index_col=0,
)

# Load the new dataset
# X_new = pd.read_csv("mriqc_learn.datasets", "SHIP183", sep='\t')

# Preprocess the new dataset
preprocessor = Pipeline(
    [
        (
            "drop_ft",
            pp.DropColumns(
                drop=[f"size_{ax}" for ax in "xyz"] + [f"spacing_{ax}" for ax in "xyz"]
            ),
        ),
        (
            "scale",
            pp.SiteRobustScaler(
                with_centering=True,
                with_scaling=True,
            ),
        ),
    ]
)
X_new_processed = preprocessor.fit_transform(X_new)

# Predict the classes of the new dataset
y_pred = model.predict(X_new_processed)